In [ ]:
#Importing all the necessary packages

import torch
import torchvision
import torchaudio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tqdm
import PIL
import sklearn
import streamlit

print("✅ All packages are imported successfully!")


In [ ]:
#GPU Information

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
if torch.cuda.is_available():
    print(f"GPU device count: {torch.cuda.device_count()}")
    print(f"GPU device name: {torch.cuda.get_device_name(0)}")

In [ ]:
#Load and Preprocess Dataset 
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for ResNet50
    transforms.ToTensor(),          # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])

# Define dataset paths
data_dir = r"D:\Projects\Dataset"

# Load datasets
train_dataset = datasets.ImageFolder(root=f"{data_dir}/Train", transform=transform)
val_dataset = datasets.ImageFolder(root=f"{data_dir}/Validation", transform=transform)
test_dataset = datasets.ImageFolder(root=f"{data_dir}/Test", transform=transform)

# Create DataLoaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print dataset sizes
print(f"✅ Training samples: {len(train_dataset)}")
print(f"✅ Validation samples: {len(val_dataset)}")
print(f"✅ Test samples: {len(test_dataset)}")

# Check class names
print(f"Classes: {train_dataset.classes}")


In [ ]:
#Load and Modify ResNet50 Model

import torch
import torchvision.models as models
import torch.nn as nn

# Load the pretrained ResNet50 model
resnet50 = models.resnet50(pretrained=True)

# Modify the final fully connected layer for binary classification
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, 2)  # 2 classes: Real & Fake

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50 = resnet50.to(device)

print("✅ ResNet50 model is ready for fine-tuning on Deepfake dataset!")


In [ ]:
#Data Augmentation & DataLoader Setup
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define data transformations
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match ResNet50 input
        transforms.RandomHorizontalFlip(),  # Augmentation for better learning
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Define dataset directory
data_dir = "D:/Projects/Dataset"

# Load datasets
datasets_dict = {
    "train": datasets.ImageFolder(root=f"{data_dir}/Train", transform=data_transforms["train"]),
    "val": datasets.ImageFolder(root=f"{data_dir}/Validation", transform=data_transforms["val"]),
    "test": datasets.ImageFolder(root=f"{data_dir}/Test", transform=data_transforms["test"])
}

# Create DataLoaders
batch_size = 32  # Adjust batch size based on available GPU memory
dataloaders = {
    "train": DataLoader(datasets_dict["train"], batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True),
    "val": DataLoader(datasets_dict["val"], batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True),
    "test": DataLoader(datasets_dict["test"], batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
}

# Check dataset sizes
print(f"✅ Training samples: {len(datasets_dict['train'])}")
print(f"✅ Validation samples: {len(datasets_dict['val'])}")
print(f"✅ Test samples: {len(datasets_dict['test'])}")
print(f"Classes: {datasets_dict['train'].classes}")  # Should output ['Fake', 'Real']

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load Pretrained ResNet50 Model
def load_resnet50():
    model = models.resnet50(weights="IMAGENET1K_V1")  # Load pretrained model
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)  # Modify last layer for binary classification (Fake vs Real)
    return model.to(device)  # Move model to GPU if available

# Initialize model
model = load_resnet50()
print("✅ Model Loaded and Modified Successfully!")


In [ ]:
#Finetuning the Model
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import time

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Loss Function & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training Function
def train_model(model, dataloaders, criterion, optimizer, epochs=5):
    start_time = time.time()

    best_val_acc = 0.0  # Track best validation accuracy
    for epoch in range(epochs):
        print(f"\n🔥 Epoch {epoch+1}/{epochs}")
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            correct = 0
            total = 0

            # Use tqdm for progress tracking
            loop = tqdm(dataloaders[phase], desc=f"{phase.upper()} [{epoch+1}/{epochs}]", leave=True)

            for images, labels in loop:
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * images.size(0)
                total += labels.size(0)
                correct += (preds == labels).sum().item()

                loop.set_postfix(loss=loss.item(), acc=100 * correct / total)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = 100 * correct / total

            print(f"✅ {phase.capitalize()} Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

            # Save best model
            if phase == 'val' and epoch_acc > best_val_acc:
                best_val_acc = epoch_acc
                torch.save(model.state_dict(), "best_model.pth")
                print("🎯 Best model saved!")

    print(f"\n🚀 Training Completed in {(time.time() - start_time) / 60:.2f} minutes!")

# Start Training
train_model(model, dataloaders, criterion, optimizer, epochs=5)


In [ ]:
#Evaluate the Model
import torch
import torch.nn as nn
from tqdm import tqdm

# Load the best trained model
model.load_state_dict(torch.load("best_model.pth"))
model.eval()  # Set model to evaluation mode

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the test function
def evaluate_model(model, dataloader):
    correct = 0
    total = 0
    running_loss = 0.0
    criterion = nn.CrossEntropyLoss()

    loop = tqdm(dataloader, desc="🔍 Evaluating on Test Set", leave=True)
    with torch.no_grad():
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            running_loss += loss.item() * images.size(0)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

            loop.set_postfix(loss=loss.item(), acc=100 * correct / total)

    test_loss = running_loss / len(dataloader.dataset)
    test_acc = 100 * correct / total
    print(f"\n🎯 Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")

# Run evaluation on the test set
evaluate_model(model, dataloaders["test"])


In [23]:
#Final code
import torch
from torchvision import transforms
from PIL import Image

# Define the transformation for a single image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the trained model
model.load_state_dict(torch.load("best_model.pth"))
model.eval()
model.to(device)

# Function to predict an image
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)

    class_names = ["Fake", "Real"]  # Adjust according to your dataset
    print(f"🔍 Prediction: {class_names[predicted.item()]}")

# Test with an image (change path accordingly)
test_image_path = r"C:\Users\mahi4\Downloads\dumb.jpeg"
predict_image(test_image_path)


C:\Users\mahi4\AppData\Local\Temp\ipykernel_7072\3171435278.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


🔍 Prediction: Real
